<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [8]:
## get the data
import wget
wget.download('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip')

'concrete_data_week4.zip'

In [18]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('concrete_data_week4.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
    zipObj.extractall()

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [36]:
# Import libraries, modules, and packages you will need. Make sure to import the preprocess_input function from keras.applications.vgg16.
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.resnet50 import preprocess_input

import numpy as np
import pandas as pd

In [20]:
# Use a batch size of 100 images for both training and validation.
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [21]:
# Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images,
# so make sure to address that when defining the ImageDataGenerator instances.
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# Train
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Validate
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [24]:
# Create a sequential model using Keras. Add VGG16 model to it and dense layer.
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

# Dense layer
model.add(Dense(num_classes, activation='softmax'))

# Don't train VGG16 model
model.layers[0].trainable = False

# see how many parameters we will need to optimize in order to train the output layer.
model.summary()

58892288/58889256 [==============================] - 7s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [25]:
# Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# Fit the model on the augmented data using the ImageDataGenerators.
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 5131s 17s/step - loss: 0.1440 - accuracy: 0.9505 - val_loss: 0.0263 - val_accuracy: 0.9934
Epoch 2/2
301/301 [==============================] - 5085s 17s/step - loss: 0.0196 - accuracy: 0.9953 - val_loss: 0.0154 - val_accuracy: 0.9965


In [27]:
model.save('classifier_VGG16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [28]:
from keras.models import load_model
resnet_model = load_model('classifier_resnet_model.h5')
vgg16_model = load_model('classifier_VGG16_model.h5')

In [29]:
data_gen = ImageDataGenerator()

In [30]:
test_datagen = data_gen.flow_from_directory(
    'concrete_data_week4/test',
    target_size = (224, 224),
    shuffle = False
)

Found 500 images belonging to 2 classes.


In [31]:
vgg = vgg16_model.evaluate_generator(test_datagen)
print(vgg)
print('Loss:', str(vgg[0]))
print('Accuracy:', str(vgg[1]))

Instructions for updating:
Please use Model.evaluate, which supports generators.
[0.17693020403385162, 0.921999990940094]
Loss: 0.17693020403385162
Accuracy: 0.921999990940094


In [32]:
resnet = resnet_model.evaluate_generator(test_datagen)
print(resnet)
print('Loss:', str(resnet[0]))
print('Accuracy:', str(resnet[1]))

[0.10667422413825989, 0.9639999866485596]
Loss: 0.10667422413825989
Accuracy: 0.9639999866485596


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [33]:
vgg_pred = vgg16_model.predict_generator(test_datagen)
res_pred = resnet_model.predict_generator(test_datagen)

Instructions for updating:
Please use Model.predict, which supports generators.


In [37]:
def predict(x):
  for i in x:
    y = np.argmax(i)
    if(y==0):
      print('Negative')
    else:
      print('Positive')

In [38]:
# first 15 predictions of ResNet50 
predict(res_pred[0:15])

Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative


In [39]:
# first 15 predictions of VGG16 
predict(vgg_pred[0:15])

Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).